<a href="https://colab.research.google.com/github/Renatocslima/Projeto_Imersao_Inteligencia_Artificial/blob/main/Projeto_Imersao_Inteligencia_Artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Consulta ao Gemine AI

In [2]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [105]:
from IPython.display import display, Markdown

In [3]:
#Configurações iniciais
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.4,
}

In [5]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [6]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [7]:
def qual_filme(filme):
  response = model.generate_content(f'''Tente descobrir qual é esse filme: {filme}
  e responda apenas com o nome dele na forma internacional junto com o ano de lançamento,
  Exemplo: Input do úsuario: "Qual o nome do filme que todo mundo corre atrás de uma galinha?",
  "City of God" ''')

  return response.text

In [97]:
def nova_sinopse(filme):
  response = model.generate_content(f'''Considerando essas informações: {filme}
  refaça a sinopse dessa obra como se fosse o apresentador do sessão da tarde,
  mas tomando o cuidado de transparecer o sentimento que o filme evoca''')

  return response.text

In [58]:
def indica_filmes(filme):
  response = model.generate_content(f'''Considerando essas informações: {filme}
  me indique 3 filmes que sejam similares a esse,
  sendo que ao menos um deles deve ser recente que 2020
  Responda apenas com os nomes em uma lista.
  Exemplo: Filme é Avatar, resposta:[Pocahontas, O Último Samurai, Duna] ''')

  return response.text

#letterboxd

In [10]:
import requests
from bs4 import BeautifulSoup

def consult_letterboxd(filme):
  url = f'https://letterboxd.com/film/{filme}/'
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get the title of the movie
  title = soup.find_all('h1')[-1].text

  # Get the release date of the movie
  release_date = soup.find('div', class_='releaseyear').text

  # Get the number of votes for the movie
  sinopse = soup.find('div', class_='truncate').text

  Resumo = {'Title': title, 'Release Date': release_date,'Sinopse': sinopse}
  return Resumo


# Que filme é esse AI?

In [124]:
def qualfilme():
  nome_do_filme = input('''Basta digitar o nome do filme ou uma lembrança do enredo e eu farei o resto.
  \nVou tentar advinhar o filme e te dar 3 sugestões de obras similares a ''')
  try:
    erro=""
    nome_do_filmeai = qual_filme(filme=nome_do_filme)
    nome_do_filmeai_split=nome_do_filmeai.split(" (")
    ano=nome_do_filmeai_split[1].replace(")","")
    nome_do_filme_link=nome_do_filmeai_split[0].replace(" ","-").replace("(",'').lower()
    info_filme = consult_letterboxd(nome_do_filme_link)
    # Verifica se é a versão mais nova ou famosa com esse nome de filme
    if info_filme['Release Date']!=ano:
      nome_do_filme_link = nome_do_filme_link+"-"+ano
      info_filme = consult_letterboxd(nome_do_filme_link)
    #Pega as indicações de fimes com base no nome e sinopse do filme escolhido
    indicacoes=indica_filmes(info_filme['Title']+" - "+info_filme['Sinopse']).replace('- ','').split('\n')
    # Gera uma nova Sinopse
    sinopse=nova_sinopse(info_filme['Title']+" - "+info_filme['Sinopse'])
    resposta = (f"\n\n**Eu acho que o filme que está falando é:** {info_filme['Title']}. \n\n")
    dicas = (f"\n\n**Filmes que também podem te interessar:** {indicacoes[0]}, {indicacoes[1]}, {indicacoes[2]}")
    resposta_final = resposta+sinopse+dicas
  except Exception as e:
    resposta_final = ("**Me desculpe!** \n*Mas infelizmente não encontrei o filme, \n por favor, tente novamente e me dê mais informações sobre a trama* \n\n")
    erro=e
  return resposta_final

#Interação com o úsuario

In [126]:
display(Markdown(qualfilme()))

Basta digitar o nome do filme ou uma lembrança do enredo e eu farei o resto.
  
Vou tentar advinhar o filme e te dar 3 sugestões de obras similares a Um grupo de amigos vai acampar na floresta e é perseguido por um assassino mascarado




**Eu acho que o filme que está falando é:** Friday the 13th. 

"Preparem-se para uma noite arrepiante, meus amigos!

Hoje, no Sessão da Tarde, vamos mergulhar no acampamento Crystal Lake, um lugar marcado por uma tragédia sombria. Um grupo de jovens e destemidos monitores está determinado a reabrir o acampamento, mas eles não estão sozinhos.

Um assassino misterioso à espreita, movido por um desejo insaciável de vingança. Cada passo que os monitores dão os aproxima do perigo, enquanto o medo se espalha como um incêndio.

Vocês sentirão arrepios na espinha e o coração na boca enquanto testemunham o terror que se desenrola em Friday the 13th. Um clássico do terror que vai deixar vocês acordados à noite, com medo de que o assassino possa estar logo atrás de vocês..."

**Filmes que também podem te interessar:** Sleepaway Camp (1983), Cabin Fever (2002), Terrifier 2 (2022)